# Aquila Optimizer Implementation for the Traveling Salesman Problem

Authors:  
Reuben Vandezande  
Sam Ferguson

In [105]:
#using Pkg
#Pkg.add("SpecialFunctions")
using SpecialFunctions
using Statistics

function calculate_g1()
    # rand motion, Eq. 16
    return 2 * rand() - 1  # Eq. 16
end

function calculate_g2(iteration::Integer, max_iterations::Integer)
    # Flight slope of aquila, Eq. 17
    return 2 * (1 - iteration / max_iterations)
end

function calculate_spiral_search(
    num_dimensions::Integer,
    r1::Float64,
    miu::Float64,
    w::Float64
)
    # Spiral search Eq.(9, 10)
    dim_list = [i for i in range(1, num_dimensions)]
    r = r1 .+ miu .* dim_list
    phi0 = 3 * pi / 2
    phi = -w .* dim_list .+ phi0
    x = r .* sin.(phi)  
    y = r .* cos.(phi)  

    return x, y
end

function calculate_quality_function(
    iteration::Integer, 
    max_iterations::Integer
)
    # Quality function, Eq.(15)
    iteration ^ ((2 * rand() - 1) / (1 - max_iterations) ^ 2)  
end

function get_levy_flight_step(
    beta::Float64
)
    # u and v are two rand variables which follow normal distribution
    # sigma_u : standard deviation of u
    sigma = gamma(1.0 + beta) * sin(pi * beta / 2) / (gamma((1 + beta) / 2.) * beta * (2. ^ ((beta - 1) / 2)))

    # levy step, Eq. 6
    step = rand() * sigma / (abs(rand()) ^ (1 / beta))
    
    return step
end

function get_best_position(
    population::Array{Float64, 2},
    fitness::Array{Float64, 1}
)
    min_index = argmin(fitness)
    return population[min_index, :], fitness[min_index]
end

function expanded_exploration(
    best_position::Array{Float64, 1},
    iteration::Integer,
    max_iterations::Integer,
    x_mean::Array{Float64, 1}
)
    return best_position * (1 - iteration / max_iterations) .+ rand() * (x_mean .- best_position)
end


function rand_individual(
    population::Array{Float64, 2},
    skip_index::Integer
)
    # Get the number of individuals in the population
    num_individuals = size(population, 1)
    
    # Filter out the skip_index
    filtered_indices = filter(x -> x != skip_index, 1:num_individuals)

    # Select a random index from the filtered array
    index = rand(filtered_indices)

    return population[index, :]
end


function narrowed_exploration(
    current_index::Integer,
    population::Array{Float64, 2},
    best_position::Array{Float64, 1},
    beta::Float64,
    r1::Float64,
    miu::Float64,
    w::Float64
)
    levy_step = get_levy_flight_step(beta)
    
    # Define spiral search
    x, y = calculate_spiral_search(size(population, 2), r1, miu, w)

    # Use Eq. 5 to compute narrowed exploration
    return best_position * levy_step .+ rand_individual(population, current_index) .+ rand() * (y - x)  
end

function expanded_exploitation(
    alpha::Float64,
    delta::Float64,
    best_position::Array{Float64, 1},
    x_mean::Array{Float64, 1},
    upper::Array{Float64, 1},
    lower::Array{Float64, 1}
)
    # Compute expanded exploitation from Eq. 13
    return alpha .* (best_position .- x_mean) - rand() .* (rand() .* (upper .- lower) .+ lower) .* delta
end

function narrowed_exploitation(
    iteration::Integer,
    max_iterations::Integer,
    individual::Array{Float64, 1},
    best_position::Array{Float64, 1},
    beta::Float64
)
    levy_step = get_levy_flight_step(beta)

    # Quality function
    qf = calculate_quality_function(iteration, max_iterations)

    # Flight slope and rand movement parameters
    g1 = calculate_g1()
    g2 = calculate_g2(iteration, max_iterations)

    # Eq. 14
    return qf .* best_position .- (g2 .* individual .* rand()) .- g2 .* levy_step .+ rand() .* g1  
end

function evolve!(
    positions,
    fitness,
    upper::Array{Float64, 1},
    lower::Array{Float64, 1},
    max_iterations:: Integer,
    alpha::Float64 = 0.1,
    delta::Float64 = 0.1,
    r1::Float64 = 10.0,
    miu::Float64 = 0.00565,
    w::Float64 = 0.005,
    beta::Float64 = 1.5
)
    # Calculate global population values
    best_position, best_fitness = get_best_position(positions, fitness)

    for iteration in 1:max_iterations
        println(best_fitness)
        # Generate a new population. Assume this happens in place?
        for idx in 1:size(positions, 1)
            # Update mean each time as individuals update positions
            x_mean = vec(mean(positions, dims=1))
    
            # Find a new position for this individual
            if iteration <= (2 / 3) * max_iterations
                if rand() < 0.5
                    pos_new = expanded_exploration(best_position, iteration, max_iterations, x_mean)
                else
                    pos_new = narrowed_exploration(idx, positions, best_position, beta, r1, miu, w)
                end
            else
                if rand() < 0.5
                    pos_new = expanded_exploitation(alpha, delta, best_position, x_mean, upper, lower)
                else
                    pos_new = narrowed_exploitation(iteration, max_iterations, positions[idx, :], best_position, beta)
                end
            end
            
            # Evaluate the position
            fitness_new = evaluate(pos_new)
            
            # If better, replace
            if fitness_new < fitness[idx]
                positions[idx, :] = pos_new
                fitness[idx] = fitness_new
                
                if fitness_new < best_fitness
                    best_position = pos_new
                    best_fitness = fitness_new                
                end
            end
        end
    end
end

evolve! (generic function with 14 methods)

In [106]:
# Sphere Function Definition
function sphere_function(x)
    return sum(x .^ 2)
end

# Define the evaluate function
function evaluate(position)
    return sphere_function(position)
end

# Initialize your search space
num_dimensions = 5  # Change this based on your problem
lower_bound = -5.0
upper_bound = 5.0
population_size = 20  # Number of search agents

# Generate initial population and fitness
positions = rand(population_size, num_dimensions) .* (upper_bound - lower_bound) .+ lower_bound
fitness = [evaluate(positions[i, :]) for i in 1:population_size]

# Define optimization parameters
max_iterations = 100

# Run the optimizer
evolve!(
    positions,
    fitness,
    fill(upper_bound, num_dimensions),
    fill(lower_bound, num_dimensions),
    max_iterations
)

18.7062490320847
3.2802879808328465
3.2659214035120696
3.1546911077820785
2.7164577677005757
2.5170937540881333
1.6665940410871394
1.4892038764198345
0.8051846485320653
0.6018814555230972
0.41583038830014585
0.36069947497864135
0.2271450569219806
0.1571993930737673
0.10285303244660832
0.05536864747670478
0.03999748587343223
0.02070945055165074
0.014531619631692156
0.004460587100651239
0.002010191064552412
0.0009295884944949332
0.0002939937911061179
0.00016872448864365403
0.00016358366015739776
0.0001273098169015706
3.959891997048275e-5
2.2740031378400667e-5
1.7157776343608955e-5
3.393957634711747e-6
1.5120637058519142e-6
6.040480767107565e-7
3.260871910913885e-7
2.620847670568552e-7
9.499309195542489e-8
2.3087477786840612e-8
5.363770744789928e-9
1.7708939879901834e-9
9.147348700358019e-10
6.47094567005417e-10
4.57485880105961e-10
5.856668370711619e-11
1.3097400581884207e-11
1.3097400581884207e-11
3.744655273884383e-12
3.744655273884383e-12
2.1848756203112618e-12
1.5923500171405318e-13
